# Imports

In [1]:
import numpy as np
from functools import partial

In [2]:
from copy import deepcopy
from functools import partial
from pprint import pprint
import os.path as osp

In [3]:
import sacred
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

from torch.backends import cudnn

In [4]:
from torch.utils.data import DataLoader, RandomSampler, BatchSampler
from typing import NamedTuple, Optional, List

In [5]:
from models.matcher import MatchERT
from models.ingredient import model_ingredient, get_model


In [6]:
from sacred import SETTINGS
from sacred.utils import apply_backspaces_and_linefeeds
from sacred import Ingredient

In [7]:
from torch.optim import Optimizer
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import DataLoader, Dataset
# from visdom_logger import VisdomLogger
from utils.metrics import *

In [8]:
from utils import pickle_load
from sacred import Experiment
from utils.data.dataset_ingredient import data_ingredient, get_loaders
from utils.data.dataset import FeatureDataset
# from utils.training import evaluate_time as evaluate
#from utils.training import evaluate
ex = sacred.Experiment('RRT Evaluation', ingredients=[data_ingredient, model_ingredient], interactive=True)

In [9]:
ex = sacred.Experiment('Prepare Top-K (VIQUAE FOR RTT)', interactive=True)
# Filter backspaces and linefeeds
SETTINGS.CAPTURE_MODE = 'sys'
ex.captured_out_filter = apply_backspaces_and_linefeeds

In [10]:
feature_name = 'r50_gldv1'
set_name = 'test'
gnd_name = 'gnd_'+ set_name+'.pkl'

In [11]:
cpu = False
cudnn_flag = 'benchmark'
temp_dir = osp.join('logs', 'temp')
resume = '/mnt/beegfs/home/smessoud/RerankingTransformer/RRT_GLD/rrt_gld_ckpts/r50_gldv1.pt'
seed = 0

In [12]:
dataset_name = 'viquae_for_rrt'
data_dir = osp.join('/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data', dataset_name)

In [13]:
device = torch.device('cuda:0' if torch.cuda.is_available() and not cpu else 'cpu')
torch.manual_seed(seed)

In [14]:
if cudnn_flag == 'deterministic':
    setattr(cudnn, cudnn_flag, True)

In [15]:
def read_file(filename):
    with open(filename) as f:
        lines = f.read().splitlines()
    return lines

In [16]:
def get_sets(desc_name, 
        train_data_dir, test_data_dir, 
        train_txt, test_txt, test_gnd_file, 
        max_sequence_len=500):
    ####################################################################################################################################
    train_lines     = read_file(osp.join(train_data_dir, train_txt))
    train_samples   = [(line.split(';;')[0], int(line.split(';;')[1]), int(line.split(';;')[2]), int(line.split(';;')[3])) for line in train_lines]
    train_set       = FeatureDataset(train_data_dir, train_samples, desc_name, max_sequence_len)
    query_train_set = FeatureDataset(train_data_dir, train_samples, desc_name, max_sequence_len)
    ####################################################################################################################################
    test_gnd_data = None if test_gnd_file is None else pickle_load(osp.join(test_data_dir, test_gnd_file))
    query_lines   = read_file(osp.join(test_data_dir, test_txt[0]))
    gallery_lines = read_file(osp.join(test_data_dir, test_txt[1]))
    query_samples   = [(line.split(';;')[0], int(line.split(';;')[1]), int(line.split(';;')[2]), int(line.split(';;')[3])) for line in query_lines]
    gallery_samples = [(line.split(';;')[0], int(line.split(';;')[1]), int(line.split(';;')[2]), int(line.split(';;')[3])) for line in gallery_lines]
    gallery_set = FeatureDataset(test_data_dir, gallery_samples, desc_name, max_sequence_len)
    query_set   = FeatureDataset(test_data_dir, query_samples,   desc_name, max_sequence_len, gnd_data=test_gnd_data)
        
    return (train_set, query_train_set), (query_set, gallery_set)

In [17]:
class MetricLoaders(DataLoader):
    train: DataLoader
    num_classes: int
    query: DataLoader
    query_train: DataLoader
    gallery: Optional[DataLoader] = None

In [18]:
class MetricLoaders(NamedTuple):
    train: DataLoader
    num_classes: int
    query: DataLoader
    query_train: DataLoader
    gallery: Optional[DataLoader] = None

In [19]:
def get_loaders(desc_name, train_data_dir, 
    batch_size=36, test_batch_size=36, 
    num_workers=8, pin_memory=True, 
    sampler='random', recalls=[1, 5, 10],
    num_candidates=100):

    (train_set, query_train_set), (query_set, gallery_set) = get_sets(desc_name, 
        train_data_dir=train_data_dir,
        test_data_dir=train_data_dir,
        train_txt='test_query.txt',
        test_txt=('test_query.txt', 'test_gallery.txt'),
        test_gnd_file='gnd_test.pkl', 
        max_sequence_len=500)

    if sampler == 'random':
        train_sampler = BatchSampler(RandomSampler(train_set), batch_size=batch_size, drop_last=False)
    elif sampler == 'triplet':
        train_nn_inds = osp.join(train_data_dir, 'nn_inds_%s.pkl'%desc_name)
        train_sampler = TripletSampler(train_set.targets, batch_size, train_nn_inds, num_candidates)
    else:
        raise ValueError('Invalid choice of sampler ({}).'.format(sampler))
    train_loader = DataLoader(train_set, batch_sampler=train_sampler, num_workers=num_workers, pin_memory=pin_memory)
    query_train_loader = DataLoader(query_train_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)
        
    query_loader   = DataLoader(query_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)
    gallery_loader = DataLoader(gallery_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)

    return MetricLoaders(train=train_loader, query_train=query_train_loader, query=query_loader, gallery=gallery_loader, num_classes=len(train_set.categories)), recalls

In [20]:
(train_set, query_train_set), (query_set, gallery_set) = get_sets('r50_gldv1',
            '/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt',
            '/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt',
            set_name+'_query.txt',
            (set_name+'_query.txt',set_name+'_gallery.txt'),
            'gnd_'+set_name+'.pkl',
            500)

In [21]:
batch_size      = 16
test_batch_size = 16
max_sequence_len = 500
sampler = 'random'
if sampler == 'random':
   train_sampler = BatchSampler(RandomSampler(train_set), batch_size=batch_size, drop_last=False)

In [22]:
num_workers = 8  # number of workers used ot load the data
pin_memory  = True  # use the pin_memory option of DataLoader 
num_candidates = 100
recalls = [1, 5, 6, 10]

In [23]:
train_loader = DataLoader(train_set, batch_sampler=train_sampler, num_workers=num_workers, pin_memory=pin_memory)
query_train_loader = DataLoader(query_train_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)

In [24]:
query_loader   = DataLoader(query_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)
gallery_loader = DataLoader(gallery_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)

In [25]:
loaders, recall_ks = get_loaders(desc_name='r50_gldv1',
    train_data_dir='/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt', 
    batch_size=36, test_batch_size=36, 
    num_workers=8, pin_memory=True, 
    sampler='random', recalls=[1, 5, 10],
    num_candidates=100)

In [26]:
model_ingredient = Ingredient('model', interactive=True)

In [27]:
def get_model(num_global_features, num_local_features, seq_len, dim_K, dim_feedforward, nhead, num_encoder_layers, dropout, activation, normalize_before):
    return MatchERT(d_global=num_global_features, d_model=num_local_features, seq_len=seq_len, d_K=dim_K, nhead=nhead, num_encoder_layers=num_encoder_layers, 
            dim_feedforward=dim_feedforward, dropout=dropout, activation=activation, normalize_before=normalize_before)

In [28]:
name = 'rrt'
num_global_features = 2048  
num_local_features = 128  
seq_len = 1004
dim_K = 256
dim_feedforward = 1024
nhead = 4
num_encoder_layers = 6
dropout = 0.0 
activation = "relu"
normalize_before = False

In [29]:
model = get_model(num_global_features,num_local_features,seq_len,dim_K,dim_feedforward,nhead,num_encoder_layers,dropout,activation,normalize_before)

In [30]:
if resume is not None:
   checkpoint = torch.load(resume, map_location=torch.device('cpu'))
   model.load_state_dict(checkpoint['state'], strict=True)

In [31]:
model.to(device)
model.eval()

MatchERT(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=1024, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Linear(in_features=1024, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.0, inplace=False)
        (dropout2): Dropout(p=0.0, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=1024, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Lin

In [32]:
loaders.query.dataset.desc_name, loaders.query.dataset.desc_name, loaders.query.dataset.data_dir

('r50_gldv1',
 'r50_gldv1',
 '/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt')

In [33]:
loaders.query

In [34]:
nn_inds_path = osp.join(loaders.query.dataset.data_dir, set_name+'_nn_inds_%s.pkl'%loaders.query.dataset.desc_name)
cache_nn_inds = torch.from_numpy(pickle_load(nn_inds_path)).long()

In [35]:
cache_nn_inds.size()

torch.Size([1257, 100])

In [36]:
num_samples, top_k = cache_nn_inds.size()
top_k = min(top_k, 100)
top_k

100

In [37]:
def evaluate(
        model: nn.Module,
        cache_nn_inds: torch.Tensor,
        query_loader: DataLoader,
        gallery_loader: DataLoader,
        recall: List[int]):
    model.eval()
    device = next(model.parameters()).device
    to_device = lambda x: x.to(device, non_blocking=True)

    query_global, query_local, query_mask, query_scales, query_positions, query_names = [], [], [], [], [], []
    gallery_global, gallery_local, gallery_mask, gallery_scales, gallery_positions, gallery_names = [], [], [], [], [], []

    with torch.no_grad():
        for entry in tqdm(query_loader, desc='Extracting query features', leave=False, ncols=80):
            q_global, q_local, q_mask, q_scales, q_positions, _, q_names = entry
            query_global.append(q_global.cpu())
            query_local.append(q_local.cpu())
            query_mask.append(q_mask.cpu())
            query_scales.append(q_scales.cpu())
            query_positions.append(q_positions.cpu())
            query_names.extend(list(q_names))

        query_global    = torch.cat(query_global, 0)
        query_local     = torch.cat(query_local, 0)
        query_mask      = torch.cat(query_mask, 0)
        query_scales    = torch.cat(query_scales, 0)
        query_positions = torch.cat(query_positions, 0)

        for entry in tqdm(gallery_loader, desc='Extracting gallery features', leave=False, ncols=80):
            g_global, g_local, g_mask, g_scales, g_positions, _, g_names = entry
            gallery_global.append(g_global.cpu())
            gallery_local.append(g_local.cpu())
            gallery_mask.append(g_mask.cpu())
            gallery_scales.append(g_scales.cpu())
            gallery_positions.append(g_positions.cpu())
            gallery_names.extend(list(g_names))
            
        gallery_global    = torch.cat(gallery_global, 0)
        gallery_local     = torch.cat(gallery_local, 0)
        gallery_mask      = torch.cat(gallery_mask, 0)
        gallery_scales    = torch.cat(gallery_scales, 0)
        gallery_positions = torch.cat(gallery_positions, 0)

        torch.cuda.empty_cache()
        evaluate_function = partial(mean_average_precision_revisited_rerank, model=model, cache_nn_inds=cache_nn_inds,
            query_global=query_global, query_local=query_local, query_mask=query_mask, query_scales=query_scales, query_positions=query_positions, 
            gallery_global=gallery_global, gallery_local=gallery_local, gallery_mask=gallery_mask, gallery_scales=gallery_scales, gallery_positions=gallery_positions, 
            ks=recall, 
            gnd=query_loader.dataset.gnd_data,
        )
        metrics = evaluate_function()
    return metrics 

In [43]:
import numpy as np
import bisect, torch
import os.path as osp
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms import functional as F

from delf import datum_io
from delf import feature_io


class FeatureDataset(Dataset):
    def __init__(self, 
            data_dir: str,
            samples: list,
            desc_name: str, 
            max_sequence_len: int,
            gnd_data=None,
            is_gallery=False):
        self.data_dir = data_dir
        self.desc_name = desc_name
        self.categories = sorted(list(set([int(entry[1]) for entry in samples])))
        self.cat_to_label = dict(zip(self.categories, range(len(self.categories))))
        self.samples = [(entry[0], self.cat_to_label[entry[1]], entry[2], entry[3]) for entry in samples]
        self.targets = [entry[1] for entry in self.samples]
        self.gnd_data = gnd_data
        self.max_sequence_len = max_sequence_len
        self.scales = [0.5, 0.70710677, 1., 1.41421354, 2., 2.82842708, 4.]
        self.is_gallery = is_gallery
  
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index, load_image=False):
        '''
        Output
            global_desc: (2048, )
            local_desc: (max_sequence_len, 128)
            local_mask: (max_sequence_len, )
            scale_inds: (max_sequence_len, )
            positions: (max_sequence_len, 2)
            label: int
            name: str
        '''
        if self.is_gallery:
            ddd
            e
            
        else:         
            image_path, label, width, height = self.samples[index]
            image_name  = osp.splitext(osp.basename(image_path))[0]
        global_path = osp.join(self.data_dir, 'delg_%s'%self.desc_name, image_name+'.delg_global')
        local_path  = osp.join(self.data_dir, 'delg_%s'%self.desc_name, image_name+'.delg_local')
        assert(osp.exists(global_path) and osp.exists(local_path))
        global_desc = datum_io.ReadFromFile(global_path)
        locations, scales, desc, attention, _ = feature_io.ReadFromFile(local_path)

        local_mask = torch.ones(self.max_sequence_len, dtype=torch.bool)
        local_desc = np.zeros((self.max_sequence_len, 128), dtype=np.float32)
        scale_inds = torch.zeros(self.max_sequence_len).long()
        seq_len = min(desc.shape[0], self.max_sequence_len)
        local_desc[:seq_len] = desc[:seq_len]
        local_mask[:seq_len] = False
        scale_inds[:seq_len] = torch.as_tensor([bisect.bisect_right(self.scales, s) for s in scales[:seq_len]]).long() - 1

        ###############################################
        # Sine embedding
        positions = torch.zeros(self.max_sequence_len, 2).float()
        normx = locations[:, 1]/float(width)
        normy = locations[:, 0]/float(height)
        positions[:seq_len] = torch.from_numpy(np.stack([normx, normy], -1)).float()[:seq_len]
        ##############################################
        
        global_desc = torch.from_numpy(global_desc).float()
        local_desc = torch.from_numpy(local_desc).float()
        
        if load_image:
            image = Image.open(osp.join(self.data_dir, image_path)).convert('RGB')
            image = image.resize((512, 512))
            return F.to_tensor(image), global_desc, local_desc, local_mask, scale_inds, positions, label, image_name
        else:
            return global_desc, local_desc, local_mask, scale_inds, positions, label, image_name

In [38]:
for entry in tqdm(query_loader, desc='Extracting query features', leave=False, ncols=80):
    print(entry)

Extracting query features:   5%|▊                | 4/79 [00:02<02:17,  1.83s/it]

[tensor([[-0.0162, -0.0013,  0.0063,  ..., -0.0145, -0.0001,  0.0082],
        [-0.0099, -0.0030, -0.0368,  ...,  0.0020, -0.0415,  0.0206],
        [-0.0171, -0.0417,  0.0097,  ...,  0.0153, -0.0200,  0.0203],
        ...,
        [-0.0114, -0.0201,  0.0056,  ..., -0.0127,  0.0148,  0.0005],
        [-0.0043, -0.0110, -0.0149,  ..., -0.0388,  0.0107,  0.0375],
        [ 0.0356, -0.0568,  0.0036,  ..., -0.0120, -0.0098, -0.0033]]), tensor([[[ 1.7209e-02,  5.5958e-02, -4.9315e-02,  ..., -2.7391e-02,
           1.1261e-01,  1.4778e-02],
         [ 7.2273e-03, -1.0292e-01,  5.4289e-02,  ..., -5.6300e-02,
           7.2462e-02,  3.1394e-02],
         [-4.1257e-03, -6.9661e-02,  1.2347e-01,  ..., -8.5311e-02,
           9.6801e-02,  5.7327e-02],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0

Extracting query features:   9%|█▌               | 7/79 [00:02<01:32,  1.29s/it]

[tensor([[ 1.8353e-02,  9.4984e-03,  1.9350e-02,  ..., -2.7219e-02,
          7.7322e-03,  3.1169e-02],
        [-1.8768e-02, -1.3033e-02, -3.5214e-02,  ...,  2.2803e-02,
         -1.0488e-02,  1.0525e-02],
        [-3.9730e-02, -3.5537e-02,  1.2602e-02,  ..., -7.5975e-05,
         -2.0404e-02, -1.9551e-02],
        ...,
        [ 1.2958e-02,  1.1159e-02, -1.8014e-02,  ..., -2.9696e-02,
          2.3081e-02, -3.4312e-02],
        [-2.3296e-02, -3.0603e-02,  2.8361e-02,  ..., -1.0669e-03,
          7.7911e-03, -2.5719e-02],
        [-2.6021e-02, -3.6260e-02, -8.4965e-03,  ..., -2.6328e-02,
          2.9091e-02,  1.7441e-02]]), tensor([[[-8.3378e-02, -8.3537e-02,  8.3029e-02,  ..., -8.7802e-02,
           4.2215e-02,  9.1555e-02],
         [-2.1533e-02, -1.1049e-03,  4.2682e-03,  ..., -7.9041e-02,
           7.4096e-02,  4.6654e-03],
         [ 1.3030e-02, -1.0785e-02,  7.6041e-02,  ..., -2.4306e-02,
           2.0201e-01,  1.5075e-01],
         ...,
         [-8.4002e-02, -1.3864e-01,  

Extracting query features:  11%|█▉               | 9/79 [00:04<01:20,  1.16s/it]

...

[tensor([[ 0.0021,  0.0161, -0.0010,  ...,  0.0529,  0.0001, -0.0020],
        [ 0.0086, -0.0272, -0.0122,  ..., -0.0331, -0.0108,  0.0312],
        [ 0.0086, -0.0272, -0.0122,  ..., -0.0331, -0.0108,  0.0312],
        ...,
        [-0.0050, -0.0335,  0.0233,  ...,  0.0025, -0.0275,  0.0344],
        [-0.0169,  0.0263,  0.0150,  ...,  0.0524,  0.0265,  0.0109],
        [-0.0150,  0.0097,  0.0237,  ..., -0.0028, -0.0190, -0.0215]]), tensor([[[-0.0587,  0.0369, -0.0783,  ...,  0.2120,  0.0758, -0.0566],
         [-0.0535,  0.0993, -0.0539,  ...,  0.1470,  0.0700,  0.0380],
         [ 0.0019,  0.0663, -0.0201,  ...,  0.1835,  0.0987, -0.0692],
         ...,
         [-0.0740, -0.0229, -0.1722,  ...,  0.0372,  0.0998,  0.0979],
         [ 0.0470, -0.0671, -0.1546,  ...,  0.0658, -0.0665,  0.1821],
         [-0.0031,  0.2130, -0.0845,  ...,  0.1298, -0.0045, -0.0045]],

        [[-0.0575, -0.1075, -0.0258,  ...,  0.1303, -0.0256, -0.1435],
         [-0.1597, -0.1349, -0.0592,  ..., -0.0142,

Extracting query features:  94%|██████████████▉ | 74/79 [00:20<00:01,  3.97it/s]

[tensor([[-0.0096, -0.0371, -0.0285,  ...,  0.0322,  0.0081,  0.0414],
        [-0.0234, -0.0165, -0.0149,  ..., -0.0092,  0.0321,  0.0207],
        [-0.0246, -0.0098,  0.0217,  ...,  0.0433,  0.0033,  0.0268],
        ...,
        [ 0.0289, -0.0197,  0.0065,  ..., -0.0201, -0.0183, -0.0493],
        [ 0.0184, -0.0078, -0.0040,  ..., -0.0027, -0.0145,  0.0243],
        [ 0.0184, -0.0078, -0.0040,  ..., -0.0027, -0.0145,  0.0243]]), tensor([[[ 0.0817, -0.0189, -0.0076,  ..., -0.0231,  0.0345,  0.0636],
         [ 0.0481, -0.0322, -0.0248,  ..., -0.0124,  0.0679,  0.0912],
         [-0.0816, -0.1463, -0.0992,  ...,  0.1024,  0.1618,  0.0184],
         ...,
         [-0.1366,  0.0222, -0.1319,  ...,  0.0217, -0.1149,  0.0038],
         [-0.0689, -0.0787, -0.0324,  ...,  0.0167, -0.0276,  0.0136],
         [ 0.0189,  0.1381, -0.0811,  ..., -0.1190,  0.1031,  0.0408]],

        [[-0.1066, -0.0780, -0.0502,  ...,  0.0929,  0.1264,  0.0546],
         [-0.0221,  0.0234, -0.0724,  ..., -0.0036,

Extracting query features:  96%|███████████████▍| 76/79 [00:20<00:00,  4.16it/s]

[tensor([[ 0.0256,  0.0178, -0.0074,  ..., -0.0040,  0.0246,  0.0223],
        [-0.0056, -0.0425, -0.0020,  ..., -0.0003, -0.0089,  0.0207],
        [ 0.0205, -0.0027,  0.0107,  ...,  0.0325,  0.0368,  0.0325],
        ...,
        [-0.0178,  0.0065, -0.0450,  ..., -0.0289, -0.0161, -0.0030],
        [ 0.0049,  0.0393,  0.0208,  ..., -0.0046,  0.0170,  0.0086],
        [-0.0042, -0.0032, -0.0034,  ...,  0.0173,  0.0237,  0.0089]]), tensor([[[-0.0108,  0.0332,  0.1039,  ...,  0.1034,  0.1023, -0.1376],
         [-0.0347, -0.1071, -0.0427,  ...,  0.1281,  0.1952, -0.1256],
         [ 0.0333, -0.0772, -0.0215,  ..., -0.1366,  0.1644,  0.0670],
         ...,
         [ 0.0330, -0.0626, -0.1078,  ..., -0.0731, -0.0083, -0.0617],
         [-0.0927,  0.0579,  0.0306,  ...,  0.0047, -0.0064, -0.1209],
         [-0.0753, -0.0076, -0.1170,  ...,  0.1225, -0.0955,  0.0271]],

        [[-0.0346, -0.0423, -0.0261,  ...,  0.1305, -0.0137,  0.0707],
         [-0.0203, -0.0257, -0.0321,  ...,  0.0283,

In [39]:
eval_function = partial(evaluate, model=model, 
        cache_nn_inds=cache_nn_inds,
        recall=recall_ks, query_loader=loaders.query, gallery_loader=loaders.gallery)

In [40]:
metrics = eval_function()

ValueError: Caught ValueError in DataLoader worker process 5.
Original Traceback (most recent call last):
  File "/mnt/beegfs/home/smessoud/anaconda3/envs/rrt/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 185, in _worker_loop
    data = fetcher.fetch(index)
  File "/mnt/beegfs/home/smessoud/anaconda3/envs/rrt/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/mnt/beegfs/home/smessoud/anaconda3/envs/rrt/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/mnt/beegfs/home/smessoud/RerankingTransformer/RRT_GLD/utils/data/dataset.py", line 55, in __getitem__
    local_desc[:seq_len] = desc[:seq_len]
ValueError: could not broadcast input array from shape (0,) into shape (0,128)


In [ ]:
pprint(metrics)

In [ ]:
use_aqe = False
aqe_params = {'k': 2, 'alpha': 0.3}

save_nn_inds = True

In [ ]:
with open(osp.join(data_dir,  set_name+'_query.txt')) as fid:
    query_lines   = fid.read().splitlines()

In [ ]:
len(query_lines)

In [ ]:
with open(osp.join(data_dir, set_name+'_gallery.txt')) as fid:
    gallery_lines = fid.read().splitlines()

In [ ]:
query_feats = []
for i in tqdm(range(len(query_lines))):
    name = osp.splitext(osp.basename(query_lines[i].split(';;')[0]))[0]
    path = osp.join(data_dir, 'delg_' + feature_name, name + '.delg_global')
    query_feats.append(datum_io.ReadFromFile(path))

In [ ]:
query_feats = np.stack(query_feats, axis=0)
query_feats = query_feats / LA.norm(query_feats, axis=-1)[:, None]

In [ ]:
query_feats.shape

In [ ]:
selection_lines = np.genfromtxt('/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt/'+
                                set_name+'_selection_imgs.txt', dtype='str')
selection_lines.shape

In [ ]:
selection_lines[0][:10]

In [ ]:
# wiki_img   = '.'.join((wiki_item['image'].split('.')[:-1]))

In [ ]:
selection_index_feats = []
for i in tqdm(range(len(selection_lines))):
    index_feats = []
    for image_file in selection_lines[i]:
        name = '.'.join((image_file.split('.')[:-1]))
        path = osp.join(data_dir, 'delg_' + feature_name, name + '.delg_global')
        index_feats.append(datum_io.ReadFromFile(path))
    selection_index_feats.append(index_feats)

In [ ]:
min([len(selection_index_feats[i]) for i in range(len(selection_index_feats))])

In [ ]:
selection_index_feats = np.array(selection_index_feats)
selection_index_feats.shape

In [ ]:
query_feats[0].shape

In [ ]:
selection_sims = []
for i in range(len(selection_index_feats)):
    index_feats = np.stack(selection_index_feats[i], axis=0)
    index_feats = index_feats / LA.norm(index_feats, axis=-1)[:, None]
    selection_sims.append(np.matmul(query_feats[i], index_feats.T))

In [ ]:
sims = np.array(selection_sims)
sims.shape

In [ ]:
sims

In [ ]:
if use_aqe:
    ## WARNING: I WAS TOO LAZY TO CORRECT IT
    ## IF YOU WANNA USE AQE PARAMATER - ADAPT THE CODE FOR VIQUAE-RRT
    alpha = aqe_params['alpha']
    nn_inds = np.argsort(-sims, -1)
    query_aug = deepcopy(query_feats)
    for i in range(len(query_feats)):
        new_q = [query_feats[i]]
        for j in range(aqe_params['k']):
            nn_id = nn_inds[i, j]
            weight = sims[i, nn_id] ** aqe_params['alpha']
            new_q.append(weight * index_feats[nn_id])
        new_q = np.stack(new_q, 0)
        new_q = np.mean(new_q, axis=0)
        query_aug[i] = new_q/LA.norm(new_q, axis=-1)
    sims = np.matmul(query_aug, index_feats.T)

In [ ]:
selection_index_feats[0].shape

In [ ]:
nn_inds = np.argsort(-sims, -1)
nn_dists = deepcopy(sims)
for i in range(query_feats.shape[0]):
    index_feats = selection_index_feats[i]
    for j in range(index_feats.shape[0]):
        nn_dists[i, j] = sims[i, nn_inds[i, j]]

In [ ]:
nn_inds.shape

In [ ]:
if save_nn_inds:
    output_path = osp.join(data_dir, set_name + '_nn_inds_%s.pkl' % feature_name)
    pickle_save(output_path, nn_inds)

In [ ]:
def compute_ap(ranks, nres):
    """
    Computes average precision for given ranked indexes.
    
    Arguments
    ---------
    ranks : zerro-based ranks of positive images
    nres  : number of positive images
    
    Returns
    -------
    ap    : average precision
    """

    # number of images ranked by the system
    nimgranks = len(ranks)

    # accumulate trapezoids in PR-plot
    ap = 0

    recall_step = 1. / nres

    for j in np.arange(nimgranks):
        rank = ranks[j]

        if rank == 0:
            precision_0 = 1.
        else:
            precision_0 = float(j) / rank

        precision_1 = float(j + 1) / (rank + 1)

        ap += (precision_0 + precision_1) * recall_step / 2.

    return ap

In [ ]:
def compute_map(ranks, gnd, kappas=[]):
    """
    Computes the mAP for a given set of returned results.

         Usage: 
           map = compute_map (ranks, gnd) 
                 computes mean average precsion (map) only
        
           map, aps, pr, prs = compute_map (ranks, gnd, kappas) 
                 computes mean average precision (map), average precision (aps) for each query
                 computes mean precision at kappas (pr), precision at kappas (prs) for each query
        
         Notes:
         1) ranks starts from 0, ranks.shape = db_size X #queries
         2) The junk results (e.g., the query itself) should be declared in the gnd stuct array
         3) If there are no positive images for some query, that query is excluded from the evaluation
    """

    map = 0.
    nq = len(gnd) # number of queries
    aps = np.zeros(nq)
    pr = np.zeros(len(kappas))
    prs = np.zeros((nq, len(kappas)))
    nempty = 0

    for i in np.arange(nq):
        qgnd = np.array(gnd[i]['ok'])
        qgndj = np.array(gnd[i]['junk'])

        # no positive images, skip from the average
        if qgnd.shape[0] == 0:
            aps[i] = float('nan')
            prs[i, :] = float('nan')
            nempty += 1
            continue

        # sorted positions of positive and junk images (0 based)
        pos  = np.arange(ranks.shape[0])[np.in1d(ranks[:,i], qgnd)]
        if len(pos) == 0:
            pos = np.array(gnd[i]['ok']).max() * np.ones_like(gnd[i]['ok'])
        
        junk = np.arange(ranks.shape[0])[np.in1d(ranks[:,i], qgndj)]

        k = 0
        ij = 0
        if len(junk):
            # decrease positions of positives based on the number of
            # junk images appearing before them
            ip = 0
            while (ip < len(pos)):
                while (ij < len(junk) and pos[ip] > junk[ij]):
                    k += 1
                    ij += 1
                pos[ip] = pos[ip] - k
                ip += 1

        # compute ap
        ap = compute_ap(pos, len(qgnd))
        map = map + ap
        aps[i] = ap

        # compute precision @ k
        pos += 1 # get it to 1-based
        for j in np.arange(len(kappas)):
            """
            if len(pos) == 0:
                max_pos = kappas[j]
            else: max_pos = max(pos)
            """
            kq = min(max(pos), kappas[j]); 
            prs[i, j] = (pos <= kq).sum() / kq
        pr = pr + prs[i, :]

    map = map / (nq - nempty)
    pr = pr / (nq - nempty)

    return map, aps, pr, prs

In [ ]:
gnd_data = pickle_load(osp.join(data_dir, gnd_name))

In [ ]:
def compute_metrics(dataset, ranks, gnd, kappas=[1, 5, 10]):
    print(ranks.shape)
    
    # old evaluation protocol
    if dataset.startswith('classic'):
        map, aps, _, _ = compute_map(ranks, gnd)
        out = {'map': np.around(map*100, decimals=3)}
        print('>> {}: mAP {:.2f}'.format(dataset, out['map']))

    # new evaluation protocol
    elif dataset.startswith('viquae'):
        
        gnd_t = []
        for i in range(len(gnd)):
            g = {}
            g['ok'] = np.concatenate([gnd[i]['hard']])
            g['junk'] = np.concatenate([gnd[i]['junk']])
            gnd_t.append(g)
        mapH, apsH, mprH, prsH = compute_map(ranks, gnd_t, kappas)


        out = {
            'H_map': np.around(mapH*100, decimals=2),
            'H_mp':  np.around(mprH*100, decimals=2),
        }

        print('>> {}: mAP H: {}'.format(dataset, out['H_map']))
        print('>> {}: mP@k{} H: {}'.format(dataset, kappas, out['H_mp']))

    return out

In [ ]:
np.arange(nn_inds.T.shape[0])[np.in1d(nn_inds.T[:,i], gnd_data['gnd'][0]['junk'])]

In [ ]:
compute_metrics('viquae', nn_inds.T, gnd_data['gnd'], kappas=[1,5,6,10])